In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    password = db.Column(db.String(100), nullable=False)

db.create_all()

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    name = data['name']
    email = data['email']
    password = data['password']

    new_user = User(name=name, email=email, password=password)
    db.session.add(new_user)
    db.session.commit()

    return jsonify({'message': 'Successfully registered!'})

@app.route('/profile', methods=['GET', 'POST', 'PUT'])
def profile():
    if request.method == 'GET':
        user = User.query.filter_by(email=request.headers['Email']).first()
        if user:
            return jsonify({'name': user.name, 'email': user.email})
        else:
            return jsonify({'message': 'User not found'}), 404
    elif request.method == 'POST':
        data = request.get_json()
        name = data['name']
        email = data['email']

        user = User.query.filter_by(email=request.headers['Email']).first()
        if user:
            user.name = name
            user.email = email
            db.session.commit()
            return jsonify({'message': 'Successfully updated profile!'})
        else:
            return jsonify({'message': 'User not found'}), 404
    elif request.method == 'PUT':
        data = request.get_json()
        password = data['password']

        user = User.query.filter_by(email=request.headers['Email']).first()
        if user:
            user.password = password
            db.session.commit()
            return jsonify({'message': 'Successfully updated password!'})
        else:
            return jsonify({'message': 'User not found'}), 404

if __name__ == '__main__':
    app.run(debug=True)